# Challenge 1 - Analyzing POS Data
Completed by Christopher Stephan

## Loading & Understanding the Data

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re
import math
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [57]:
#adjust the display settings to be able to better understand our data
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)  # Show all rows (if needed)

### POS Data

In [58]:
pos_data = pd.read_csv("POS_data.csv")
super_data = pd.read_csv("supermarket_POS_data.csv")

In [59]:
pos_data.head()

,Date,Time,Transaction,Item
0,10/30/2016,9:58:11,1,Bread
1,10/30/2016,10:05:34,2,Scandinavian
2,10/30/2016,10:05:34,2,Scandinavian
3,10/30/2016,10:07:57,3,Hot chocolate
4,10/30/2016,10:07:57,3,Jam


In [60]:
#Extracting the total rows and columns in our pos dataset
pos_data.shape

(21293, 4)

In [61]:
pos_data.dtypes

Date           object
Time           object
Transaction     int64
Item           object
dtype: object

In [62]:
# changing the date and time to a datetime object 
pos_data['datetime'] = pd.to_datetime(pos_data['Date'].astype(str) + ' ' + pos_data['Time'].astype(str))

In [63]:
#removing the date and time columns as they are now redundant 
pos_data = pos_data.drop(columns = ['Date', 'Time'], axis=1)

In [64]:
pos_data.dtypes

Transaction             int64
Item                   object
datetime       datetime64[ns]
dtype: object

In [65]:
#Checking if any of the rows are duplicated in the dataset
print(f'There are {pos_data.duplicated().sum()} duplicated rows in our pos dataset.')

There are 1653 duplicated rows in our pos dataset.


In [66]:
#Rename columns for better readability
pos_data = pos_data.rename(columns={
    'Transaction': 'transaction',
    'Item': 'item',
})

#Confirm the column names are updated
pos_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21293 entries, 0 to 21292
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   transaction  21293 non-null  int64         
 1   item         21293 non-null  object        
 2   datetime     21293 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 499.2+ KB


In [67]:
pos_data.isna().mean()

transaction    0.0
item           0.0
datetime       0.0
dtype: float64

**The dataset is complete, no missing values**

In [68]:
#Summary statistics for integer & float features
pos_data.describe()

,transaction,datetime
count,21293.000000,21293
mean,4951.990889,2017-01-17 14:54:19.669234176
min,1.000000,2016-10-30 09:58:11
25%,2548.000000,2016-12-03 14:07:07
50%,5067.000000,2017-01-21 12:34:58
75%,7329.000000,2017-02-28 10:58:04
max,9684.000000,2017-04-09 15:04:24
std,2787.758400,NaN


### SuperMarket Data

In [69]:
super_data.head()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


In [70]:
#Extracting the total rows and columns in our supermarket dataset
super_data.shape

(1000, 17)

In [71]:
super_data.dtypes

Invoice ID                  object
Branch                      object
City                        object
Customer type               object
Gender                      object
Product line                object
Unit price                 float64
Quantity                     int64
Tax 5%                     float64
Total                      float64
Date                        object
Time                        object
Payment                     object
cogs                       float64
gross margin percentage    float64
gross income               float64
Rating                     float64
dtype: object

In [72]:
#Checking if any of the rows are duplicated in the dataset
print(f'There are {super_data.duplicated().sum()} duplicated rows in our pos dataset.')

There are 0 duplicated rows in our pos dataset.


In [73]:
#changing the date and time to a datetime object as well
super_data['datetime'] = pd.to_datetime(super_data['Date'].astype(str) + ' ' + super_data['Time'].astype(str))

In [74]:
#removing the date and time columns as they are now redundant 
super_data = super_data.drop(columns = ['Date', 'Time'], axis=1)

In [75]:
#Rename columns for better readability
super_data.columns = [col.replace(" ", "_").lower() for col in super_data.columns]

In [76]:
super_data.columns

Index(['invoice_id', 'branch', 'city', 'customer_type', 'gender',
       'product_line', 'unit_price', 'quantity', 'tax_5%', 'total', 'payment',
       'cogs', 'gross_margin_percentage', 'gross_income', 'rating',
       'datetime'],
      dtype='object')

In [77]:
super_data.isna().mean()

invoice_id                 0.0
branch                     0.0
city                       0.0
customer_type              0.0
gender                     0.0
product_line               0.0
unit_price                 0.0
quantity                   0.0
tax_5%                     0.0
total                      0.0
payment                    0.0
cogs                       0.0
gross_margin_percentage    0.0
gross_income               0.0
rating                     0.0
datetime                   0.0
dtype: float64

**The dataset is complete, no missing values**

In [78]:
#Summary statistics for integer & float features
super_data.describe()

,unit_price,quantity,tax_5%,total,cogs,gross_margin_percentage,gross_income,rating,datetime
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1.000000e+03,1000.000000,1000.00000,1000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905e+00,15.379369,6.97270,2019-02-14 15:30:27.480000
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905e+00,0.508500,4.00000,2019-01-01 10:39:00
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905e+00,5.924875,5.50000,2019-01-24 17:58:45
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905e+00,12.088000,7.00000,2019-02-13 17:37:00
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905e+00,22.445250,8.50000,2019-03-08 15:29:30
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905e+00,49.650000,10.00000,2019-03-30 20:37:00
std,26.494628,2.923431,11.708825,245.885335,234.17651,6.131498e-14,11.708825,1.71858,NaN


### Creating Dataframes

In [79]:
pos = pos_data.copy()
pos.set_index('datetime', inplace=True)

In [80]:
#addding date-related columns for visuals later 
pos['year'] = pos.index.year
pos['month'] = pos.index.month_name()
pos['day_of_month'] = pos.index.day
pos['weekday'] = pos.index.day_name()
pos['date'] = pos.index.date
pos['hour_of_day'] = pos.index.hour

In [81]:
super = super_data.copy()
super.set_index('datetime', inplace=True)

In [82]:
#addding date-related columns for visuals later 
super['day_of_month'] = super.index.day
super['month'] = super.index.month_name()
super['weekday'] = super.index.day_name()
super['date'] = super.index.date
super['hour_of_day'] = super.index.hour

In [83]:
#putting transaction as the index of the original pos dataset as it is already a unique identifier
pos_data.set_index('transaction', inplace=True)  

In [84]:
#putting the invoice number as the index 
super_data.set_index('invoice_id', inplace=True) 

## Exploratory Data Analysis

### Time Series Analysis

In [85]:
pos.head(10)

,transaction,item,year,month,day_of_month,weekday,date,hour_of_day
datetime,,,,,,,,
2016-10-30 09:58:11,1,Bread,2016,October,30,Sunday,2016-10-30,9
2016-10-30 10:05:34,2,Scandinavian,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:05:34,2,Scandinavian,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:07:57,3,Hot chocolate,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:07:57,3,Jam,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:07:57,3,Cookies,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:08:41,4,Muffin,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:13:03,5,Coffee,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:13:03,5,Pastry,2016,October,30,Sunday,2016-10-30,10


In [86]:
super.head(5)

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,total,payment,cogs,gross_margin_percentage,gross_income,rating,day_of_month,month,weekday,date,hour_of_day
datetime,,,,,,,,,,,,,,,,,,,,
2019-01-05 13:08:00,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,Ewallet,522.83,4.761905,26.1415,9.1,5,January,Saturday,2019-01-05,13
2019-03-08 10:29:00,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,Cash,76.40,4.761905,3.8200,9.6,8,March,Friday,2019-03-08,10
2019-03-03 13:23:00,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,Credit card,324.31,4.761905,16.2155,7.4,3,March,Sunday,2019-03-03,13
2019-01-27 20:33:00,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,Ewallet,465.76,4.761905,23.2880,8.4,27,January,Sunday,2019-01-27,20
2019-02-08 10:37:00,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,Ewallet,604.17,4.761905,30.2085,5.3,8,February,Friday,2019-02-08,10


In [87]:
# Sales trend over time
fig1 = px.line(pos.groupby('date').size().reset_index(), x='date', y=0,
               title="Sales Trend Over Time", labels={'date': 'Date', 0: 'Number of Transactions'})
fig1.show()

# Hourly sales pattern
fig2 = px.bar(pos.groupby('hour_of_day').size().reset_index(), x='hour_of_day', y=0,
              title="Transactions by Hour of Day", labels={'hour_of_day': 'Hour', 0: 'Transaction Count'})
fig2.show()

# Weekday sales pattern
fig3 = px.bar(pos.groupby('weekday').size().reset_index(), x='weekday', y=0,
              title="Transactions by Weekday", labels={'weekday': 'Weekday', 0: 'Transaction Count'})
fig3.show()


In [88]:
# Count transactions for each hour and weekday
heatmap_data = pos.groupby(['weekday', 'hour_of_day']).size().reset_index(name="count")

# Create heatmap
fig4 = px.density_heatmap(heatmap_data, x='hour_of_day', y='weekday', z='count', 
                          title="Transaction Density: Hour vs. Weekday",
                          labels={'hour_of_day': 'Hour of Day', 'weekday': 'Weekday', 'count': 'Transaction Count'},
                          color_continuous_scale='Blues')
fig4.show()

In [89]:
# Average spending per hour
hourly_spending = super.groupby('hour_of_day')['total'].mean().reset_index()

# Plot spending trend
fig5 = px.line(hourly_spending, x='hour_of_day', y='total', markers=True,
               title="Average Spending per Hour",
               labels={'hour_of_day': 'Hour', 'total': 'Avg Spending per Transaction'})
fig5.show()

In [90]:
fig6 = px.box(super, x='weekday', y='total',
              title="Daily Sales Distribution",
              labels={'weekday': 'Weekday', 'total': 'Sales Amount'})
fig6.show()

#### Insights from Time Series of data

First in the **Sales Trend Over Time** we could clearly see that there was a peak occuring on a regular basis, in this case weekly. Further investigation indicated that this peak actually occurs every single saturday, confirmed by the **Sales by weekday** bar graph. 

Next in the **Transactions by hour of Day** graph we could see that most people were shopping around midday, with the most occuring around 12 pm. This was expected, but also surprising because I expected there to be a peak in the night time as well, especially after work. 

The rest of the graphs in this section are confirming what was stated above, with the **Transaction Density: Hour vs Weekday** heatmap summarizing brilliantly these insights. 



### Branch Analysis

In [91]:
pos.head()

,transaction,item,year,month,day_of_month,weekday,date,hour_of_day
datetime,,,,,,,,
2016-10-30 09:58:11,1,Bread,2016,October,30,Sunday,2016-10-30,9
2016-10-30 10:05:34,2,Scandinavian,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:05:34,2,Scandinavian,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:07:57,3,Hot chocolate,2016,October,30,Sunday,2016-10-30,10
2016-10-30 10:07:57,3,Jam,2016,October,30,Sunday,2016-10-30,10


In [92]:
super.head()

,invoice_id,branch,city,customer_type,gender,product_line,unit_price,quantity,tax_5%,total,payment,cogs,gross_margin_percentage,gross_income,rating,day_of_month,month,weekday,date,hour_of_day
datetime,,,,,,,,,,,,,,,,,,,,
2019-01-05 13:08:00,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,Ewallet,522.83,4.761905,26.1415,9.1,5,January,Saturday,2019-01-05,13
2019-03-08 10:29:00,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,Cash,76.40,4.761905,3.8200,9.6,8,March,Friday,2019-03-08,10
2019-03-03 13:23:00,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,Credit card,324.31,4.761905,16.2155,7.4,3,March,Sunday,2019-03-03,13
2019-01-27 20:33:00,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,Ewallet,465.76,4.761905,23.2880,8.4,27,January,Sunday,2019-01-27,20
2019-02-08 10:37:00,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,Ewallet,604.17,4.761905,30.2085,5.3,8,February,Friday,2019-02-08,10


In [97]:
# Sales per branch
fig7 = px.bar(super.groupby('branch').size().reset_index(), x='branch', y=0,
              title="Number of Transactions per branch", labels={'branch': 'branch', 0: 'Transaction Count'})
fig7.show()

# Revenue per branch
fig8 = px.bar(super.groupby('branch')['total'].sum().reset_index(), x='branch', y='total',
              title="total revenue per branch", labels={'branch': 'branch', 'total': 'total revenue'})
fig8.show()


In [98]:
# Aggregate total sales by branch and weekday
branch_weekday_sales = super.groupby(['branch', 'weekday'])['total'].sum().reset_index()

# Grouped bar chart
fig9 = px.bar(branch_weekday_sales, x='weekday', y='total', color='branch',
              title="Branch Performance by Weekday",
              labels={'weekday': 'Weekday', 'total': 'Total Sales', 'branch': 'Branch'},
              barmode='group')
fig9.show()

In [99]:
# Aggregate sales by product line per branch
product_sales = super.groupby(['branch', 'product_line'])['total'].sum().reset_index()

# Stacked bar chart
fig10 = px.bar(product_sales, x='branch', y='total', color='product_line',
              title="Best-Selling Product Lines by Branch",
              labels={'branch': 'Branch', 'total': 'Total Sales', 'product_line': 'Product Line'},
              barmode='stack')
fig10.show()

In [100]:
# Aggregate customer type count per branch
customer_type_distribution = super.groupby(['branch', 'customer_type'])['invoice_id'].nunique().reset_index()

# Stacked bar chart
fig11 = px.bar(customer_type_distribution, x='branch', y='invoice_id', color='customer_type',
              title="Customer Type Distribution by Branch",
              labels={'branch': 'Branch', 'invoice_id': 'Number of Transactions', 'customer_type': 'Customer Type'},
              barmode='stack')
fig11.show()

In [101]:
# Aggregate total sales per branch by payment method
branch_payment = super.groupby(['branch', 'payment'])['total'].sum().reset_index()

# Stacked bar chart
fig12 = px.bar(branch_payment, x='branch', y='total', color='payment',
              title="Payment Method Usage by Branch",
              labels={'branch': 'Branch', 'total': 'Total Sales', 'payment': 'Payment Method'},
              barmode='stack')
fig12.show()

In [102]:
# Calculate average transaction value per branch
branch_avg_spending = super.groupby('branch')['total'].mean().reset_index()

# Bar chart
fig13 = px.bar(branch_avg_spending, x='branch', y='total', color='branch',
              title="Average Spending Per Transaction by Branch",
              labels={'branch': 'Branch', 'total': 'Average Spending'})
fig13.show()

In [103]:
# Average rating per branch
branch_ratings = super.groupby('branch')['rating'].mean().reset_index()

# Bar chart for ratings
fig14 = px.bar(branch_ratings, x='branch', y='rating', color='branch',
               title="Average Customer Rating by Branch",
               labels={'branch': 'Branch', 'rating': 'Average Rating'})
fig14.show()

#### Insights from Branch Analysis of data

This section did not really give us much information about the branches, simply because all kinds of analysis yielded a very similar result. It seems that all three branches perform roughly the same in terms of sales, products being sold (although some are more home/life focused and others sports focused), and the users shopping there are more or less similar, using the same types of payment methods, having the same credit scores, etc. There are more or less equal amounts of member and normal users in all branches. 

### Customer Analysis